In [ ]:
import os

import tensorflow as tf
import pytest
import numpy as np
import keras.backend as K
from keras.models import Sequential, Model , load_model
from keras.layers import Input, Dense, Flatten
from keras.layers import Conv1D, MaxPool1D, Cropping1D, UpSampling1D
from keras.layers import ZeroPadding1D, AveragePooling1D, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPool2D, Cropping2D, UpSampling2D
from keras.layers import ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D
from numpy import random

from kerassurgeon import operations
from kerassurgeon import utils
from kerassurgeon import Surgeon
from kerassurgeon.utils import get_inbound_nodes, get_outbound_nodes


In [ ]:
train_dir = 'C:/Users/asus/Desktop/images/train'
validation_dir = 'C:/Users/asus/Desktop/images/validation'

In [ ]:
batch_size = 16
epochs = 30
warmup_epocks = 2
learning_rate = 0.0005
warmup_learning_rate = 0.0001
height = 128
width = 128
colors = 3
n_classes = 5
es_patience = 5
rlrop_patience = 3
decay_drop = 0.5
based_model_last_block_layer_number = 5 

In [ ]:
    model = models.Sequential()
    model.add(layers.Conv2D(32,(3,3), activation='relu',input_shape=(height, width, colors)))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    #model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(32,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    #model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(64,(3,3),activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(n_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.001)))


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
model.load_weights('model_weights.h5')

In [ ]:
w1 = model.layers[0].get_weights()[0]
w2 = model.layers[3].get_weights()[0]
w3 = model.layers[6].get_weights()[0]

conv_layer_weights = [w1,w2,w3]

for i in range(len(conv_layer_weights)):
    weight=conv_layer_weights[i]
    weights_dict = {}
    num_filters = len(weight[0,0,0,:])

    for j in range (num_filters):
        w_s = np.sum(abs(weight[:,:,:,j]))
        filt = 'filt_{}'.format(j)
        weights_dict[filt] = w_s
    weights_dict_sort = sorted(weights_dict.items(), key=lambda kv: kv[1])
    print('L1 norm conv layer {} \n'.format(i+1),weights_dict_sort)

    weights_value=[]
    for elem in weights_dict_sort:
        weights_value.append(elem[1])
    xc=range(num_filters)



    plt.figure(i+1,figsize=(7,5))
    plt.plot(xc,weights_value)
    plt.xlabel('filter_num')
    plt.ylabel('l1 norm')
    plt.title(' conv layer {}'.format(i+1))
    plt.grid(True)

    plt.style.use['classic']

In [ ]:
from kerassurgeon import indentify

In [ ]:
from kerassurgeon.operations import delete_channels,delete_layer



In [ ]:
layer_0 = model.layers[0]
model_new = delete_channels(model,layer_0,[7,26,30,15,24])

model_new.compile(loss='categorical_crossentropy' , optimizer='rmsprop',metrics['accuracy'])

In [ ]:
layer_6 = model.layer[6]
model_new = delete_channels(model,layer_6,[7,26,30,15,24])


In [ ]:
model_new.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics['accuracy'])


In [ ]:
from kerassurgeon import Surgeon

layer_0 = model.layer[0]
layer_6 = model.layer[6]

surgeon = Surgeon(model)

surgeon.add_job('delete_channels',layer_0,channels=[7,26,30,15,24])
surgeon.add_job('delete_channels',layer_6,channels=[9,62,1,15,24])

model_new = surgeon.operate()
model_new.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics['accuracy'])


